In [1]:
#       Copyright 2021
#       Abhirup Ghosh <abhirup.ghosh.184098@gmail.com>,
#
#       This program is free software; you can redistribute it and/or modify
#       it under the terms of the GNU General Public License as published by
#       the Free Software Foundation; either version 2 of the License, or
#       (at your option) any later version.
#
#       This program is distributed in the hope that it will be useful,
#       but WITHOUT ANY WARRANTY; without even the implied warranty of
#       MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#       GNU General Public License for more details.
#
#       You should have received a copy of the GNU General Public License
#       along with this program; if not, write to the Free Software
#       Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston,
#       MA 02110-1301, USA.

In [2]:
import numpy as np
import pandas as pd
from pycbc.waveform import get_fd_waveform
from pycbc  import  detector
from bilby_pipe import xml_converter 
import matplotlib.pyplot as plt
import pycbc.filter.matchedfilter as mfilter
import pycbc.psd
import pycbc.noise.gaussian
import pycbc.types.frequencyseries
from pycbc.types import TimeSeries, FrequencySeries, zeros

import lal, lalinference, numpy as np
import lalinference.imrtgr.imrtgrutils as tgr
import lalinference.imrtgr.nrutils as nr

In [3]:
def calc_isco_freq(a):
        """ 
        Calculate the ISCO frequency of a Kerr BH as a function of the Kerr parameter 
        
        a : Kerr parameter (numpy array) 
        """

        r_isco = nr.calc_isco_radius(a)
        u_isco = r_isco**-0.5
        v_isco = u_isco*(1.-a*u_isco**3.+a**2.*u_isco**6)**(1./3.)
        return v_isco**3./np.pi

In [4]:
data_dict = {'gw150914': np.genfromtxt("/Users/abhirupghosh/Documents/Work/spinqnm/runs/nonGR/GW150914-like/domega220_dtau220_0p0_GR_widerdist/GW150914-like_nonGR_0p0.dat"),
             'gw190521': np.genfromtxt("/Users/abhirupghosh/Documents/Work/spinqnm/runs/nonGR/GW190521-like/domega220_dtau220_0p0_GR_widerdist/GW190521-like_nonGR_0p0.dat"),
             'sxsbbh1066': np.genfromtxt("/Users/abhirupghosh/Documents/Work/spinqnm/runs/NRinjections/SXS_BBH_0166/hwinjcbc_1126259456.xml.dat")}

In [5]:
df = 1./4
fit_formula = 'bbh_average_fits_precessing'

for (flow, fhigh, event) in zip([20., 11., 25.], [1024., 256., 2048.], ['gw150914', 'gw190521', 'sxsbbh1066']):

    mass_1, mass_2, luminosity_distance, psi, phase, geocent_time, ra, dec, theta_jn, a_1, a_2, tilt_1, tilt_2, phi_12, phi_jl = data_dict[event]
    
    Mf, af = tgr.calc_final_mass_spin(mass_1, mass_2, a_1, a_2, a_1*np.cos(tilt_1), a_2*np.cos(tilt_2), phi_12, fit_formula)
    f_isco_Kerr = calc_isco_freq(af)/(Mf*lal.MTSUN_SI)

    hpf, hcf = get_fd_waveform(approximant="IMRPhenomPv2",
                                mass1=mass_1, mass2=mass_2, 
                                spin1z=a_1*np.cos(tilt_1), spin2z=a_2*np.cos(tilt_2),
                                inclination=theta_jn,
                                coa_phase=phase,
                                distance=luminosity_distance,
                                delta_f=df,
                                f_lower=8.)

    psd = pycbc.psd.aLIGOZeroDetHighPower(len(hpf), df, flow)

    snr_sq_tot = 0
    snr_sq_pre = 0
    snr_sq_post = 0
    
    if event == 'sxsbbh1066':
        det_list = ["H1", "L1", "V1"]
    else:
        det_list = ["H1", "L1"]

    for det in det_list:
        d = pycbc.detector.Detector(det)
        Fp,Fc = d.antenna_pattern(right_ascension=ra, 
                                declination=dec, 
                                polarization=psi,
                                t_gps=geocent_time, 
                                polarization_type='tensor')
        signal = Fp*hpf+Fc*hcf
        signal_freq = pycbc.types.frequencyseries.FrequencySeries(signal,delta_f=df,dtype=complex)
        SNR_tot = mfilter.sigma(signal_freq,psd=psd,low_frequency_cutoff=flow,high_frequency_cutoff=fhigh)
        SNR_pre = mfilter.sigma(signal_freq,psd=psd,low_frequency_cutoff=flow,high_frequency_cutoff=f_isco_Kerr)
        SNR_post = mfilter.sigma(signal_freq,psd=psd,low_frequency_cutoff=f_isco_Kerr,high_frequency_cutoff=fhigh)
        
        snr_sq_tot += SNR_tot**2.
        snr_sq_pre += SNR_pre**2.
        snr_sq_post += SNR_post**2.

    print(f"{f_isco_Kerr:.2f} & {np.sqrt(snr_sq_tot):.2f} & {np.sqrt(snr_sq_pre):.2f} & {np.sqrt(snr_sq_post):.2f}")    

130.92 & 25.20 & 22.22 & 11.88
31.91 & 19.56 & 7.90 & 17.90
73.55 & 70.85 & 57.68 & 41.15
